In [1]:
import plotly.express as px
import polars as pl

from udonpred_benchmarking.constants import DATA_DIR
from udonpred_benchmarking.plots import METHODS

In [3]:
DATA_DIR

PosixPath('/home/julius/projects/University/Publications/UdonPred/Benchmarking/data')

In [4]:
protein_df = pl.read_csv(DATA_DIR / "per_protein_performance.csv")
protein_stats = pl.read_csv(DATA_DIR / "per_protein_stats.csv")

In [12]:
heatmap_df = (
    protein_df
    .join(protein_stats, on="ID")
    .filter(pl.col("metric") == "spearman")
    .drop(["metric", "pscore_mean", "pscore_mean_category_thirds", "pscore_mean_quartile", "len"])
    .sort(pl.col("value").mean().over("ID"))
    #.sort("ID")
    .pivot(on="ID", index="method", values="value")
    .to_pandas()
)

heatmap_df.set_index("method", inplace=True)
fig = px.imshow(heatmap_df, x=heatmap_df.columns, y=heatmap_df.index, labels={"x": "Protein", "y": "Method", "color": "Spearman"}, aspect="auto")
fig.update_layout(height=500)
fig.update_xaxes(showticklabels=False)
fig.show()

In [14]:
pcords_df = (
    protein_df
    .join(protein_stats, on="ID")
    .filter(pl.col("metric") == "spearman")
    .drop("metric")
    .pivot(on="method", values="value")
)

fig1 = px.parallel_coordinates(
    pcords_df,
    color="pscore_mean",
    dimensions=METHODS,
    labels={"x": "Method", "y": "Spearman", "color": "avg. G-score"}
)
fig1.show()
fig2 = px.parallel_coordinates(
    pcords_df,
    color="pscore_mean_quartile",
    dimensions=METHODS,
    labels={"x": "Method", "y": "Spearman", "color": "avg. G-score"}
)
fig2.show()